Synthplan!
==========

Okay, a few more hours of this work, then back to Drake for the day.

Where we left off
-----------------

We've got a system where a block with thrusters at one end is moving in a 2d space:

In [1]:
import math
import time

from pydrake.systems.framework import (DiagramBuilder,)
from pydrake.systems.primitives import (ConstantVectorSource,)
from pydrake.geometry import (Box,
                              DrakeVisualizer)
from pydrake.math import (RigidTransform, RollPitchYaw)
from pydrake.multibody.plant import (AddMultibodyPlantSceneGraph, CoulombFriction, Propeller, PropellerInfo)
from pydrake.multibody.tree import (SpatialInertia, RotationalInertia, PlanarJoint)

from pydrake.systems.analysis import Simulator

def make_box(mbp, name):
    inertia = SpatialInertia.MakeFromCentralInertia(1, [0, 0, 0], RotationalInertia(1/600, 1/120, 1/120))
    body = mbp.AddRigidBody(name, inertia)
    shape = Box(1, 0.1, 0.1)
    mbp.RegisterVisualGeometry(
        body=body, X_BG=RigidTransform(), shape=shape, name=f"{name}_visual",
        diffuse_color=[1., 0.64, 0.0, 0.5])
    body_friction = CoulombFriction(static_friction=0.6,
                                    dynamic_friction=0.5)
    mbp.RegisterCollisionGeometry(
        body=body, X_BG=RigidTransform(), shape=shape,
        name="{name}_collision", coulomb_friction=body_friction)
    return body

def make_mbp(builder):
    mbp, sg = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    robot_body = make_box(mbp, "robot")
    planar_joint = mbp.AddJoint(
        PlanarJoint(name="robot_planar_joint",
                    frame_on_parent=mbp.world_frame(),
                    frame_on_child=robot_body.body_frame(),
                    damping=[1,1,0.9]))
    
    blocker1 = make_box(mbp, "blocker1")
    mbp.WeldFrames(A=mbp.world_frame(), B=blocker1.body_frame(), X_AB=RigidTransform([1,0.5,0]))
    return mbp, sg

def add_thrusters(builder, mbp):
    robot_body_index = mbp.GetBodyByName("robot").index()
    robot_tail_forward = RigidTransform(p=[-0.5, 0, 0], rpy=RollPitchYaw([math.pi/2, 0, 0]))
    robot_tail_clockwise = RigidTransform(p=[-0.5, 0, 0], rpy=RollPitchYaw([math.pi/2, 0, math.pi/2]))
    thrusters = builder.AddSystem(
        Propeller([PropellerInfo(robot_body_index, X_BP=robot_tail_forward),
                   PropellerInfo(robot_body_index, X_BP=robot_tail_clockwise)]))
    builder.Connect(thrusters.get_spatial_forces_output_port(), mbp.get_applied_spatial_force_input_port())
    builder.Connect(mbp.get_body_poses_output_port(), thrusters.get_body_poses_input_port())
    return thrusters

def run(sleeptime=0.001, thrust = [10, 2]):
    builder = DiagramBuilder()
    mbp, sg = make_mbp(builder)
    mbp.Finalize()
    DrakeVisualizer.AddToBuilder(builder, sg)

    thrusters = add_thrusters(builder, mbp)
    cmd = builder.AddSystem(ConstantVectorSource(thrust))
    builder.Connect(cmd.get_output_port(), thrusters.get_command_input_port())
    
    diagram = builder.Build()
    simulator = Simulator(diagram)
    for step in range(1000):
        simulator.AdvanceTo(0.001 * step)
        mbp_context = diagram.GetSubsystemContext(mbp, simulator.get_context())
        time.sleep(sleeptime)

run()

... and we've got an ability to forward-simulate to check where a force schedule leaves the block ...

In [2]:
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial

import numpy as np

experiment_count = 0

def experiment(start_state, force_schedule, sleeptime=None):
    global experiment_count
    experiment_count += 1
    if experiment_count % 10 == 0:
        print(f"{experiment_count} experiments run so far...")
    
    builder = DiagramBuilder()
    mbp, sg = make_mbp(builder)
    mbp.Finalize()
    DrakeVisualizer.AddToBuilder(builder, sg)

    thrusters = add_thrusters(builder, mbp)

    breaks = [0]
    for _, t in force_schedule:
        breaks.append(breaks[-1] + t)
    forces = np.array([f for f, t in force_schedule] + [force_schedule[-1][0]])
    force_traj = PiecewisePolynomial.ZeroOrderHold(breaks, forces.transpose())
    controller = builder.AddSystem(TrajectorySource(force_traj))
    builder.Connect(controller.get_output_port(), thrusters.get_command_input_port())
    
    diagram = builder.Build()
    simulator = Simulator(diagram)
    mbp_context = diagram.GetSubsystemContext(mbp, simulator.get_context())
    mbp.SetPositionsAndVelocities(mbp_context, start_state)
    for step in range(int(1000 * breaks[-1])):
        simulator.AdvanceTo(0.001 * step)
        mbp_context = diagram.GetSubsystemContext(mbp, simulator.get_context())
        if sleeptime: time.sleep(sleeptime)
    return mbp.GetPositionsAndVelocities(diagram.GetSubsystemContext(mbp, simulator.get_context()))

experiment([0, 0, 0, 0, 0, 0], [[[10, 2], 2], [[0, 0], 2]], sleeptime=1e-3)

array([  1.85452250e+000,   9.72879021e-001,   6.14681308e+000,
         6.69750184e-004,   6.66424001e-002,   7.93285551e-100])

... That lets us define a "Primitive" as a force schedule that takes us from a bounding box of start conditions to a bounding box of end conditions ...

In [3]:
import copy

class Primitive:
    def __init__(self, start_min, start_max, end_min, end_max, force_schedule):
        self.start_min = start_min
        self.start_max = start_max
        self.end_min = end_min
        self.end_max = end_max
        self.force_schedule=force_schedule  # List of [forces, duration]

def ensemble(start_min, start_max):
    coords = [[]]
    for axis in range(len(start_min)):
        new_coords = []
        for coord in coords:
            new_coords += [coord + [start_min[axis]]]
            new_coords += [coord + [start_max[axis]]]
        coords = new_coords
    return coords

def cost(start_min, start_max, force_schedule, target):
    min_time = min(t for f, t in force_schedule)
    max_force = max(math.sqrt(fx * fx + fy * fy) for (fx, fy), t in force_schedule)
    duration = sum(t for f, t in force_schedule)
    if min_time < 0.001: return float('inf')
    if min_time < 0.1: return 2 / min_time
    if max_force > 20: return 2 * max_force
    starts = ensemble(start_min, start_max)
    results = (experiment(e, force_schedule) for e in starts)
    distances = (np.linalg.norm(np.array(result) - np.array(target)) for result in results)
    return duration + (1 / min_time) + max_force + sum(distances) / len(starts)

s = cost(start_min=[0, 0, 0, 0, 0, 0], start_max=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1], force_schedule=[[[0, 1], 1], [[0, -1], 1]], target=[0, 0, 1, 0, 0, 0])
print(s)

10 experiments run so far...
20 experiments run so far...
30 experiments run so far...
40 experiments run so far...
50 experiments run so far...
60 experiments run so far...
5.14881005636


Today
-----

Today begins here.  We would like to create a "synthesizer" that tries to build a Primitive between two points. We'll start with an easy one.

In [ ]:
from scipy.optimize import minimize

def flatten_force_schedule(schedule):
    result = []
    for forces, time in schedule:
        for fx, fy in forces:
            result.append(fx, fy, time)
    return np.array(result)

def unflatten_force_schedule(schedule):
    result = []
    assert schedule.size % 3 == 0
    for i in range(schedule.size // 3):
        result.append([[schedule[i*3], schedule[i*3+1]], schedule[i*3+2]])
    return result

def naive_synthesis(start_min, start_max, target, initial_guess = [0, 0, 1, 0, 0, 1, 0, 0, 1]):
    def cost_fun(guess):
        print(unflatten_force_schedule(guess))
        costval = cost(start_min, start_max, unflatten_force_schedule(guess), target)
        print(costval)
        return costval
    min = minimize(fun=cost_fun, x0=initial_guess)
    print(min)

naive_synthesis(start_min=[0, 0, 0, 0, 0, 0], start_max=[0.1, 0.1, 0.1, 0.1, 0.1, 0.1], target=[0, 0, 1, 0, 0, 0])

[[[0.0, 0.0], 1.0], [[0.0, 0.0], 1.0], [[0.0, 0.0], 1.0]]
70 experiments run so far...
80 experiments run so far...
90 experiments run so far...
100 experiments run so far...
110 experiments run so far...
120 experiments run so far...
4.96447459594
[[[1.4901161193847656e-08, 0.0], 1.0], [[0.0, 0.0], 1.0], [[0.0, 0.0], 1.0]]
130 experiments run so far...
140 experiments run so far...
150 experiments run so far...


...which crashes the kernel with a buffer overflow.  Just like two weeks ago, I'm fetching up on a Drake bug...